# **Методы**

In [ ]:
import scipy.io
import scipy.stats
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import copy
from tqdm import tqdm
from matplotlib.patches import Rectangle
from typing import List, Tuple
from matplotlib import rc
import matplotlib.animation as animation
try:
    from pycpd import RigidRegistration
except ImportError:
    !pip install pycpd
    from pycpd import RigidRegistration

rc('animation', html='jshtml')
matplotlib.rcParams['animation.embed_limit'] = 2**256

In [2]:
def generate_room(room_size_x, room_size_y, obstacles=[]):
    """
    Создает пустую комнату со стенами размера (room_size_x, room_size_y) и дополнительными препятствиями (obstacles)
    """
    # Заполняем комнату 1 (белые клетки)
    room = np.ones((room_size_x+1, room_size_y+1))

    # Заполняем стены как 0 (черные клетки)
    room[0, :] = 0
    room[-1,:] = 0
    room[:, 0] = 0
    room[:,-1] = 0

    # Заполняем препятствия как 0 (черные клетки)
    for obstacle in obstacles:
        x, y = obstacle
        room[x:x+1, y:y+1] = 0

    return room

def scan_room(room, room_size_x, room_size_y, position, max_depth, angle_step):
    """
    Сканирует комнату (room) размера (room_size_x, room_size_y) из заданной позиции робота (position). Лучи сканера имеют максимальную глубину (max_depth) и шаг поворота лазера (angle_step)
    """
    # Стартовая позиция робота и его угол поворота
    x, y, radian = position
    degress_start = math.degrees(radian)

    # Массив углов сканов
    angles = np.arange(degress_start, degress_start + 360, angle_step) % 360

    # Сканирование комнаты
    scan_distances = []
    for angle in angles:
        radian_angle = np.radians(angle)

        distance = -1
        for depth in range(1, max_depth):
            x_pixel = int(x + depth * np.cos(radian_angle))
            y_pixel = int(y + depth * np.sin(radian_angle))
            if room_size_x <= x_pixel < 0 or room_size_y <= y_pixel < 0 or room[x_pixel,y_pixel]==0:
              distance = depth
              break

        scan_distances.append(distance)

    return np.array(scan_distances), angles

def plot_room_and_scan(pose, scan_distances, angles, room: np.ndarray, obstacle_positions, plot, id):
    """
    Создает график для отображения комнаты (room) и лучей всех сканера
    """
    plot.imshow(room.T, cmap='gray', origin='lower')

    angles = np.radians(angles)

    # Отрисовка выпущенных луч скана
    for i, (angle, distance) in enumerate(zip(angles, scan_distances)):
        if distance != -1:
            end_point = (pose[0] + distance * np.cos(angle), pose[1] + distance * np.sin(angle))
            color = 'blue' if i == 0 else 'green'
            linewidth = 2.0 if i == 0 else 0.8
            plot.plot([pose[0], end_point[0]], [pose[1], end_point[1]], color=color, linewidth=linewidth)

    # Отрисовка робота на карте
    plot.scatter(pose[0], pose[1], color='blue', label='Робот', s=100)
    plot.set_xlabel('X')
    plot.set_ylabel('Y')
    plot.set_title(f'Результат сканирования {id}')
    #plot.legend()
    plot.grid(True)



def scan_to_point_cloud(scan_distances):
    """
    Создает облако точек
    """
    angles_rad = np.radians(np.arange(0, 360, angle_step))

    # Фильтрация массива дистанций
    valid_distances = scan_distances[scan_distances > 0]
    valid_angles_rad = angles_rad[scan_distances > 0]
    # Вычисление локальных координат препятствий (облако точек)
    x_coords = valid_distances * np.cos(valid_angles_rad)
    y_coords = valid_distances * np.sin(valid_angles_rad)

    # Возвращаем координаты точек в облаке
    return np.column_stack((x_coords, y_coords))



def plot_point_clouds(A, B, plot, transformed_A=None):
    """
    Создает график для отображения облаков точек из обоих сканирований и трансформированного облака (для кластера графиков)
    """
    #plot.figure(figsize=plot_size)

    plot.scatter(A[:, 0], A[:, 1], color='red', marker='^', label='Облако точек 1')
    plot.scatter(B[:, 0], B[:, 1], color='blue', marker='o', label='Облако точек 2')
    if transformed_A is not None:
        plot.scatter(transformed_A[:, 0], transformed_A[:, 1], color='green', marker='x', label='Трансформированое облако точек 2')

    plot.set_xlabel("X")
    plot.set_ylabel("Y")
    plot.set_title("Визуализация облаков точек")
    plot.legend()
    plot.grid(True)
    plot.axis('equal')
    #plt.show()

def plot_point_clouds_2(A, B, transformed_A=None):
    """
    Создает график для отображения облаков точек из обоих сканирований и трансформированного облака (для одного графика)
    """
    plt.figure(figsize=plot_size)

    plt.scatter(A[:, 0], A[:, 1], color='red', marker='^', label='Облако точек 1')
    plt.scatter(B[:, 0], B[:, 1], color='blue', marker='o', label='Облако точек 2')
    if transformed_A is not None:
        plt.scatter(transformed_A[:, 0], transformed_A[:, 1], color='green', marker='x', label='Трансформированое облако точек 2')

    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Визуализация облаков точек")
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

def transformed_point(point_cloud, point_cloud1):
    """
    Рассчитывает трансформаированное облако точек и параметров трансформации из облаков точек первого и второго сканирований
    """
    #point_cloud /= np.max(np.abs(point_cloud))
    #point_cloud1 /= np.max(np.abs(point_cloud1))

    reg = RigidRegistration(X=point_cloud, Y=point_cloud1)
    TY, (s_reg, R_reg, t_reg) = reg.register()

    return reg.transform_point_cloud(Y=point_cloud1), (s_reg, R_reg, t_reg)

def print_scan_result(room_size_x, room_size_y, scanner_position, scanner_position1, transformation):
    """
    Расчитывает вычисленную трансформацию, а также ее ошибку
    """
    cos_theta = np.round(transformation[1][0,0], ROUND_DECIMALS)

    #print(transformation[1][0,1])
    #print(transformation[1][1,0])

    # Нахождение знака для угла
    estimated_theta_sign = 1
    if transformation[1][0,1] < 0:
        estimated_theta_sign = -1
    #elif transformation[1][1,0] <= 0:
    elif transformation[1][1,0] > 0:
        estimated_theta_sign = 1

    #print(estimated_theta_sign)

    # Параметры предсказанной трансформации
    cpd_estimated_theta = np.round(math.acos(cos_theta) * estimated_theta_sign, ROUND_DECIMALS)
    cpd_estimated_x_translation = np.round(transformation[2][0], ROUND_DECIMALS)
    cpd_estimated_y_translation = np.round(transformation[2][1], ROUND_DECIMALS)

    # Действительная трансформация
    actual_x_translation = np.round(scanner_position1[0] - scanner_position[0], ROUND_DECIMALS)
    actual_y_translation = np.round(scanner_position1[1] - scanner_position[1], ROUND_DECIMALS)
    actual_theta_translation = np.round(scanner_position1[2] - scanner_position[2], ROUND_DECIMALS)

    # Дельта трансформаций между истенным и предсказанным
    #diff_x = np.round(abs(actual_x_translation - cpd_estimated_x_translation), ROUND_DECIMALS)
    #diff_y = np.round(abs(actual_y_translation - cpd_estimated_y_translation), ROUND_DECIMALS)
    #diff_theta = np.round(abs(actual_theta_translation - cpd_estimated_theta), ROUND_DECIMALS)
    diff_x = np.round(actual_x_translation - cpd_estimated_x_translation, ROUND_DECIMALS)
    diff_y = np.round(actual_y_translation - cpd_estimated_y_translation, ROUND_DECIMALS)
    diff_theta = np.round(actual_theta_translation - cpd_estimated_theta, ROUND_DECIMALS)

    # Процентное отклонение
    diff_x_from_room = np.round(diff_x / room_size_x * 100, ROUND_DECIMALS)               # Процентное отклонение по Х, где 100% - размер комнаты по X
    diff_y_from_room = np.round(diff_y / room_size_y * 100, ROUND_DECIMALS)               # Процентное отклонение по Y, где 100% - размер комнаты по Y
    diff_theta_from_2pi = np.round(diff_theta / math.radians(360) * 100, ROUND_DECIMALS)  # Процентное отклонение угла поворота, где 100% - 360 градусов (2пи)

    # Оценка вектора трансформации
    translation_vector_length = np.round(math.sqrt((diff_x)**2 + (diff_y)**2), ROUND_DECIMALS)
    room_diagonal_length = np.round(math.sqrt(room_size_x**2 + room_size_y**2), ROUND_DECIMALS)
    translation_vector_accuracy = np.round(translation_vector_length / room_diagonal_length * 100, ROUND_DECIMALS)

    # Предсказанные координаты сканеры после трансформации
    #translated_scanner_x = np.round(scanner_position[0] + cpd_estimated_x_translation, ROUND_DECIMALS)
    #translated_scanner_y = np.round(scanner_position[1] + cpd_estimated_y_translation, ROUND_DECIMALS)
    #translated_scanner_rad = np.round(scanner_position[2] + cpd_estimated_theta, ROUND_DECIMALS)
    #translated_scanner_position = (translated_scanner_x, translated_scanner_y, translated_scanner_rad)
    translated_scanner_x = np.round(scanner_position[0] + diff_x, ROUND_DECIMALS)
    translated_scanner_y = np.round(scanner_position[1] + diff_y, ROUND_DECIMALS)
    translated_scanner_rad = np.round(scanner_position[2] + diff_theta, ROUND_DECIMALS)
    translated_scanner_position = (translated_scanner_x, translated_scanner_y, translated_scanner_rad)

    # Печать результатов
    print(f"{room_size_x=} {room_size_y=} {angle_step=}")
    print(f"{scanner_position=} {scanner_position1=}\n{translated_scanner_position=}")
    print(f"Точность трансформации ветора X/Y: {100 - translation_vector_accuracy}%")
    data = {
        "Истинное": [actual_x_translation, actual_y_translation, actual_theta_translation],
        "Рассчитанное": [cpd_estimated_x_translation, cpd_estimated_y_translation, cpd_estimated_theta],
        "Разница": [diff_x, diff_y, diff_theta],
        "Разница%": [f"({diff_x_from_room}%)", f"({diff_y_from_room}%)", f"({diff_theta_from_2pi}%)"]
    }
    print(pd.DataFrame(data, index=["X", "Y", "θ(rad)"]))

    return translated_scanner_position

def Scan_experiment(room_size_x, room_size_y, obstacles, scanner_position, scanner_position1):
    """
    Функция эксперимента по поиску местоположения исходя из сканирования окружения
    """
    # Генерация комнаты
    room = generate_room(room_size_x, room_size_y, obstacles)

    #Заготовка для двух горизонтальных графиков
    fig, (plot1, plot2) = plt.subplots(1, 2, figsize=(plot_size[0]*3, plot_size[1]*3))
    # Сканирование комнаты в позиции 1
    scan_distances, angles = scan_room(room, room_size_x, room_size_y, scanner_position, max_depth, angle_step)
    point_cloud = scan_to_point_cloud(scan_distances)
    plot_room_and_scan(scanner_position, scan_distances, angles, room, obstacles, plot1, 1)
    # Сканирование комнаты в позиции 2
    scan_distances1, angles1 = scan_room(room, room_size_x, room_size_y, scanner_position1, max_depth, angle_step)
    point_cloud1 = scan_to_point_cloud(scan_distances1)
    plot_room_and_scan(scanner_position1, scan_distances1, angles1, room, obstacles, plot2, 2)
    # Вывод двух горизонтальных графиков
    plt.show()

    # Получение трансформаированного облака и параметров трансформации
    point_cloud_transformed, transformation = transformed_point(point_cloud, point_cloud1)
    # Отображения облака точек
    plot_point_clouds_2(point_cloud, point_cloud1, point_cloud_transformed)

    # Отображение результата сканировния
    print_scan_result(room_size_x, room_size_y, scanner_position, scanner_position1, transformation)

**Метод CPD (Coherent Point Drift) (Когерентный дрейф точек)** - это алгоритм машинного обучения, предназначенный для кластеризации данных. Он основан на идее, что каждая точка данных принадлежит определенному кластеру, и кластеры имеют свои центры.
Метод CPD позволяет обрабатывать данные с различными распределениями и шума, так как он учитывает расстояния между точками и центрами кластеров.

In [3]:
def turn_on_angle(scanner_positions=[(0,0,math.radians(0))], rad_angle=math.radians(90), steps=3):
    """
    Выполняет добавление точек движения по углу
    """
    step = rad_angle / steps
    current_scanner_positions = scanner_positions[len(scanner_positions)-1]

    for i in range(steps):
        scanner_positions.append((current_scanner_positions[0], current_scanner_positions[1], current_scanner_positions[2]+(step*(i+1))))

    return scanner_positions

def move_on_xDist(scanner_positions=[(0,0,math.radians(0))], distance=5, steps=5):
    """
    Выполняет добавление точек движения по оси X
    """
    step = distance / steps
    current_scanner_positions = scanner_positions[len(scanner_positions)-1]

    for i in range(steps):
        scanner_positions.append((current_scanner_positions[0]+(step*(i+1)), current_scanner_positions[1], current_scanner_positions[2]))

    return scanner_positions

def move_on_yDist(scanner_positions=[(0,0,math.radians(0))], distance=5, steps=5):
    """
    Выполняет добавление точек движения по оси Y
    """
    step = distance / steps
    current_scanner_positions = scanner_positions[len(scanner_positions)-1]

    for i in range(steps):
        scanner_positions.append((current_scanner_positions[0], current_scanner_positions[1]+(step*(i+1)), current_scanner_positions[2]))

    return scanner_positions

def percent_difference(a, b):
    """
    Выполняет рассчет процентной разности между числами
    """
    return np.round(float(abs( (a - b) / ((a + b) / 2) ) * 100), ROUND_DECIMALS)

class Map():
    """
    Класс, представляющий сетку занятости (occupancy grid map), используемую для отслеживания вероятности наличия препятствий в окружающей среде на основе данных лазерного сканирования.
    """
    def __init__(self, xsize, ysize, grid_size):
        # Установка размеров сетки с учетом границ
        self.xsize = xsize+2
        self.ysize = ysize+2
        self.grid_size = grid_size
        # Установка всех значений в матрице занятости нулями
        self.log_prob_map = np.zeros((self.xsize, self.ysize))

        self.alpha = 1.0            # Предполагаемая толщина препятствий
        self.beta = 5.0*np.pi/180.0 # Предполагаемая ширина лазерного луча
        self.z_max = 150.0          # Максимальное расстояние измерения лазера

        # Предварительное выделение памяти для координат всех ячеек в 3D тензоре (pre-allocation = скорость, быстрота, успех, богатство, влияние, власть, господство, поступок гигачада)
        self.grid_position_m = np.array([np.tile(np.arange(0, self.xsize*self.grid_size, self.grid_size)[:,None], (1, self.ysize)),
                                         np.tile(np.arange(0, self.ysize*self.grid_size, self.grid_size)[:,None].T, (self.xsize, 1))])

        # Логарифм вероятности для добавления или удаления из карты
        self.l_occ = np.log(0.65/0.35)
        self.l_free = np.log(0.35/0.65)

    def update_map(self, pose, scan_distances, angles):
        """
        Реализует метод обновления карты `update_map`. В ходе обновления происходит вычисление направлений от робота к каждой ячейке, а также определение свободных и занятых ячеек на основе данных лазерного сканирования.
        Вероятности в каждой ячейке корректируются согласно измерениям.

        Принимает:
        - позицию робота (pose).
        - измеренные расстояния (scan_distances).
        - углы лазерного сканирования (angles).
        """
        dx = self.grid_position_m.copy()  # Тензор координат всех ячеек
        dx[0, :, :] -= pose[0]            # Матрица всех x-координат ячеек
        dx[1, :, :] -= pose[1]            # Матрица всех y-координат ячеек
        theta_to_grid = np.arctan2(dx[1, :, :], dx[0, :, :])  # Матрица всех направлений от робота к ячейке
        #theta_to_grid = np.arctan2(dx[1, :, :], dx[0, :, :]) - pose[2]

        # Оборачиваем углы в диапазон +pi / - pi
        #theta_to_grid[theta_to_grid > np.pi] -= 2. * np.pi
        #theta_to_grid[theta_to_grid < -np.pi] += 2. * np.pi
        theta_to_grid = np.mod(theta_to_grid, 2 * np.pi)

        # Матрица L2-расстояний до всех ячеек от робота
        dist_to_grid = scipy.linalg.norm(dx, axis=0)

        # Для каждого лазерного луча
        for r, b in zip(scan_distances, angles):
            # Рассчитываем, какие ячейки рассчитаны как свободные или занятые, чтобы знать, какие ячейки обновить
            free_mask = (np.abs(theta_to_grid - b) <= self.beta/2.0) & (dist_to_grid < (r - self.alpha/2.0))
            occ_mask = (np.abs(theta_to_grid - b) <= self.beta/2.0) & (np.abs(dist_to_grid - r) <= self.alpha/2.0)

            # Корректируем значения в ячейках соответственно
            self.log_prob_map[occ_mask] += self.l_occ
            self.log_prob_map[free_mask] += self.l_free

    def compute_free_area(self):
        """
        Реализует метод поиска свободной площади `compute_free_area`. Рассчитывает свободную от препятствий площадь у построенной карты.
        """
        # Определение свободных ячеек (Отрицательные значения соответствуют свободным ячейкам)
        free_mask = (self.log_prob_map < 0)

        # Вычисление общей свободной площади
        free_area = np.sum(free_mask) * self.grid_size**2

        return free_area

In [4]:
def SLAM(room_size_x, room_size_y, obstacles, scanner_positions, plotting):
    """
    Функция реализующая алгоритм построения карты с помощью перемещающегося по ней робота со сканером.
    """
    # Генерация комнаты
    room = generate_room(room_size_x, room_size_y, obstacles)
    # Генерация карты занятости
    map = Map(int(room_size_x/grid_size), int(room_size_y/grid_size), grid_size)

    scan_distances=[]
    angles=[]
    point_clouds=[]
    point_clouds_transformed=[]
    transformations=[]
    translated_scanner_position=[]
    poses = []
    log_prob_maps = []

    plt.figure(figsize=plot_size)
    plt.imshow(room.T, cmap='gray', origin='lower')
    # Добавление точек маршрута робота
    for x, y, angle in scanner_positions:
        plt.arrow(x, y, 0.1 * np.cos(angle), 0.1 * np.sin(angle), head_width=0.6, head_length=1, fc='red', ec='red')
    plt.grid(True)
    plt.title('Предстоящий путь робота')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()
    print("\nКоличество предстоящих сканов:", len(scanner_positions))

    # Сканирование комнаты в каждой позиции
    for i, scanner_position in enumerate(scanner_positions):
      print("\nШаг:",i)

      # Сканирование
      scan_distance, angle = scan_room(room, room_size_x, room_size_y, scanner_position, max_depth, angle_step)
      scan_distances.append(scan_distance)
      angles.append(angle)

      point_clouds.append(scan_to_point_cloud(scan_distances[i]))


      if i >= 1:
        # Получение трансформированного облака и параметров трансформации
        point_cloud_transformed, transformation = transformed_point(point_clouds[i-1], point_clouds[i])

        # Отображение результата сканировния и получение вычисленного положения
        #translated_scanner_position.append(print_scan_result(room_size_x, room_size_y, scanner_positions[i-1], scanner_positions[i], transformation))
        translated_scanner_position.append(print_scan_result(room_size_x, room_size_y, translated_scanner_position[i-1], scanner_positions[i], transformation))

        # Обновление карты
        map.update_map(translated_scanner_position[i], scan_distances[i], np.radians(angles[i])) # update the map
        log_prob_maps.append(copy.deepcopy(map.log_prob_map)) # Сохраняем map в список для создания gif
        pose = [translated_scanner_position[i][0]/grid_size, translated_scanner_position[i][1]/grid_size, translated_scanner_position[i][2]]
        poses.append(pose) # Сохраняем pose в список для создания gif

        if (plotting):
            # Заготовка для четырех графиков
            fig, ((plot0, plot1), (plot2, plot3)) = plt.subplots(2, 2, figsize=(plot_size[0]*3, plot_size[1]*3))

            # Вывод графиков сканирований
            plot_room_and_scan(scanner_positions[i-1], scan_distances[i-1], angles[i-1], room, obstacles, plot0, 1)
            plot_room_and_scan(scanner_positions[i], scan_distances[i], angles[i], room, obstacles, plot1, 2)

            # Вывод графика облака точек
            plot_point_clouds(point_clouds[i-1], point_clouds[i], plot2, point_cloud_transformed)

            # Вывод графика текущего состояния построения карты
            circle = plt.Circle((pose[0], pose[1]), radius=1/grid_size, fc='y')
            plt.gca().add_patch(circle)
            #arrow = pose[0:2] + np.array([0, 2./grid_size]).dot(np.array([[np.sin(pose[2]), np.cos(pose[2])], [np.cos(pose[2]), -np.sin(pose[2])]]))
            arrow = pose[0:2] + np.array([0, 2./grid_size]).dot(np.array([[np.sin(pose[2]), np.cos(pose[2])], [np.cos(pose[2]), np.sin(pose[2])]]))
            plot3.plot([pose[0], arrow[0]], [pose[1], arrow[1]])
            plot3.imshow((1.0 - 1./(1.+np.exp(map.log_prob_map))).T, 'Greys', origin='lower', extent=[0, map.log_prob_map.shape[0], 0, map.log_prob_map.shape[1]])
            plot3.set_title('Результат построения карты')
            plt.show()

      elif i == 0:
        # Добавление исходной точки в путь робота
        translated_scanner_position.append(scanner_position)

        # Обновление карты
        map.update_map(scanner_position, scan_distances[i], np.radians(angles[i]))
        pose = [scanner_position[0]/grid_size, scanner_position[1]/grid_size, scanner_position[2]]

        if (plotting):
            plt.figure(figsize=plot_size)
            circle = plt.Circle((pose[0], pose[1]), radius=1/grid_size, fc='y')
            plt.gca().add_patch(circle)
            arrow = pose[0:2] + np.array([0, 2./grid_size]).dot(np.array([[np.sin(pose[2]), np.cos(pose[2])], [np.cos(pose[2]), np.sin(pose[2])]]))
            plt.plot([pose[0], arrow[0]], [pose[1], arrow[1]])
            plt.imshow((1.0 - 1./(1.+np.exp(map.log_prob_map))).T, 'Greys', origin='lower', extent=[0, map.log_prob_map.shape[0], 0, map.log_prob_map.shape[1]])
            plt.show()


    # Вывод итоговых графиков
    fig, ((plot0, plot1), (plot2, plot3)) = plt.subplots(2, 2, figsize=(plot_size[0]*3, plot_size[1]*3))

    # График 0
    #plot0.imshow(room.T, cmap='gray', origin='lower', extent=[0, room.shape[1], 0, room.shape[0]])
    plot0.imshow(room.T, cmap='gray', origin='lower')
    plot0.grid(True)
    plot0.set_title('Комната')
    plot0.set_xlabel('X')
    plot0.set_ylabel('Y')

    # График 1
    #plot1.imshow(room.T, cmap='gray', origin='lower', extent=[0, room.shape[1], 0, room.shape[0]])
    plot1.imshow(room.T, cmap='gray', origin='lower')
    # Добавление точек маршрута робота
    for x, y, angle in translated_scanner_position:
        plot1.arrow(x, y, 0.1 * np.cos(angle), 0.1 * np.sin(angle), head_width=0.6, head_length=1, fc='red', ec='red')
    plot1.grid(True)
    plot1.set_title('Путь робота')
    plot1.set_xlabel('X')
    plot1.set_ylabel('Y')

    # График 2
    #plot2.imshow((1.0 - 1./(1.+np.exp(map.log_prob_map))).T, 'Greys', origin='lower', extent=[0, map.log_prob_map.shape[1], 0, map.log_prob_map.shape[0]])
    plot2.imshow((1.0 - 1./(1.+np.exp(map.log_prob_map))).T, 'Greys', origin='lower')
    plot2.grid(True)
    plot2.set_title('Probability')
    plot2.set_xlabel('X')
    plot2.set_ylabel('Y')

    # График 3
    #plot3.imshow((1.0 - 1./(1.+np.exp(map.log_prob_map))).T, 'Greys', origin='lower', extent=[0, map.log_prob_map.shape[1], 0, map.log_prob_map.shape[0]])
    #plot3.imshow(map.log_prob_map.T, 'Greys', origin='lower', extent=[0, map.log_prob_map.shape[1], 0, map.log_prob_map.shape[0]])
    plot3.imshow((1.0 - 1./(1.+np.exp(map.log_prob_map))).T, 'Greys', origin='lower')
    plot3.imshow(map.log_prob_map.T, 'Greys', origin='lower')
    plot3.grid(True)
    plot3.set_title('Log Probabilities')
    plot3.set_xlabel('X')
    plot3.set_ylabel('Y')

    plt.tight_layout()
    plt.show()

    def update(frame):
        plot0.clear()
        plot1.clear()
        plot2.clear()
        plot3.clear()

        # Логика обновления данных для каждого кадра
        scan_distance, angle = scan_room(room, room_size_x, room_size_y, scanner_positions[frame], max_depth, angle_step)

        # Обновление графиков сканирования
        plot_room_and_scan(scanner_positions[frame-1], scan_distances[frame-1], angles[frame-1], room, obstacles, plot0, frame)
        plot_room_and_scan(scanner_positions[frame], scan_distances[frame], angles[frame], room, obstacles, plot1, frame)

        # Обновление облаков точек
        plot_point_clouds(point_clouds[frame - 1], point_clouds[frame], plot2, point_cloud_transformed)

        # Обновление состояния карты в plot3
        #plot3.clear()  # Очищаем предыдущие данные на plot3
        pose = poses[frame-1]
        circle = plt.Circle((pose[0], pose[1]), radius=1/grid_size, fc='y')
        plot3.add_patch(circle)
        arrow = pose[0:2] + np.array([0, 2./grid_size]).dot(np.array([[np.sin(pose[2]), np.cos(pose[2])], [np.cos(pose[2]), np.sin(pose[2])]]))
        plot3.plot([pose[0], arrow[0]], [pose[1], arrow[1]])
        plot3.imshow((1.0 - 1./(1.+np.exp(log_prob_maps[frame-1]))).T, 'Greys', origin='lower', extent=[0, log_prob_maps[frame-1].shape[0], 0, log_prob_maps[frame-1].shape[1]])
        plot3.set_title('Результат построения карты')

        return plot0, plot1, plot2, plot3

    # Создание анимации
    anim = animation.FuncAnimation(fig, update, frames=len(scanner_positions), repeat=True)
    plt.show()  # Для отображения в интерактивном режиме

    # Вывод численных результатов
    print("\nКоличество сканов:", len(scanner_positions))
    room_size = room_size_x * room_size_y
    print("Площадь пустой комнаты:", room_size)
    room_size_obstacles = room_size - len(obstacles)
    print("Площадь заполненной комнаты:", room_size_obstacles)
    room_size_estimated = map.compute_free_area()
    print("Площадь найденная:", room_size_estimated)
    room_size_percent = np.round(float(room_size_estimated / room_size_obstacles * 100), ROUND_DECIMALS)
    print("Найденная площадь относительно действительной:", room_size_percent, "%")
    percent_diff = percent_difference(room_size_obstacles, room_size_estimated)
    print("Точность найденной площади:", 100 - percent_diff, "%")
    print("Ошибка найденной площади:  ", percent_diff, "%")

    return anim

Класс "Map" является реализацией метода Occupancy Grid Mapping представленного в главе 9 "Вероятностной робототехники" Себастьяна Труна (Sebastian Thrun) и др.
- Книга: https://github.com/literator1996/veroyatnostnaya_robototehnica/blob/master/src/9/1-9-ready2final-inedit.pdf
- Код: https://gist.github.com/superjax/33151f018407244cb61402e094099c1d

# **SLAM**

In [5]:
# DEFINE CONSTANTS
plot_size = (5,5)                           # Размер рисунков при отрисовке
max_depth = 200                             # Максимальная длина луча сканера
angle_step = 3                              # Шаг угла в градусах
ROUND_DECIMALS = 4                          # Для округления чисел с плавающей точкой
#grid_size = 1.00                            # Размер сетки карты занятости
grid_size = 0.25

In [ ]:
room_size_x = 30
room_size_y = 30

obstacles = []
for i in range(0, 12):
    for j in range(10, 21):
        obstacles.append((i,j))
for i in range(19, 30):
    for j in range(10, 21):
        obstacles.append((i,j))
obstacles.append((5,21))
obstacles.append((25,21))

scanner_positions = [(15, 5,math.radians(0))]
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, 20, 40)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, -10, 20)


#scanner_positions = [(5, 5,math.radians(0))]
#scanner_positions = move_on_xDist(scanner_positions, 20, 20)
#scanner_positions = turn_on_angle(scanner_positions, math.radians(180), 6)
#scanner_positions = move_on_xDist(scanner_positions, -10, 10)
#scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
#scanner_positions = move_on_yDist(scanner_positions, 20, 20)
#scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
#scanner_positions = move_on_xDist(scanner_positions, -10, 10)
#scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
#scanner_positions = move_on_xDist(scanner_positions, 20, 20)

anim = SLAM(room_size_x, room_size_y, obstacles, scanner_positions, True)

In [ ]:
anim

In [8]:
writervideo = animation.FFMpegWriter(fps=10)
anim.save('animation_room1(H).mp4', writer=writervideo)

In [ ]:
room_size_x = 30
room_size_y = 30

obstacles = []

for l in range (0, 19):
    for i in range (0, 19 - l):
        obstacles.append((i, l))
    for i in range (room_size_x - 20 + l, room_size_x):
        obstacles.append((i, room_size_x - l))

scanner_positions = [(2, 27,math.radians(0))]

for i in range (0, 6):
    scanner_positions = move_on_xDist(scanner_positions, 4, 4)
    scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
    scanner_positions = move_on_yDist(scanner_positions, -4, 4)
    scanner_positions = turn_on_angle(scanner_positions, math.radians(90))

anim = SLAM(room_size_x, room_size_y, obstacles, scanner_positions, False)

In [ ]:
anim

In [11]:
writervideo = animation.FFMpegWriter(fps=10)
anim.save('animation_room2(-).mp4', writer=writervideo)

In [ ]:
room_size_x = 30
room_size_y = 30

obstacles = []

for i in range (0, 5):
    for j in range (0, 5):
        obstacles.append((5 + i, 5 + j))
        obstacles.append((20 + i, 5 + j))
        obstacles.append((5 + i, 20 + j))
        obstacles.append((20 + i, 20 + j))

scanner_positions = [(17, 27,math.radians(0))]

scanner_positions = move_on_xDist(scanner_positions, 12, 4)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, -24, 8)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_xDist(scanner_positions, -24, 8)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, 24, 8)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_xDist(scanner_positions, 12, 4)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, -12, 4)

anim = SLAM(room_size_x, room_size_y, obstacles, scanner_positions, False)

In [ ]:
anim

In [14]:
writervideo = animation.FFMpegWriter(fps=4)
anim.save('animation_room3(::)_notaccurate.mp4', writer=writervideo)

In [ ]:
room_size_x = 50
room_size_y = 50

obstacles = []

for i in range (0, 5):
    for j in range (0, 5):
        obstacles.append((15 + i, 15 + j))
        obstacles.append((30 + i, 15 + j))
        obstacles.append((15 + i, 30 + j))
        obstacles.append((30 + i, 30 + j))

scanner_positions = [(25, 45,math.radians(0))]
scanner_positions = move_on_xDist(scanner_positions, 23, 23)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, -40, 40)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_xDist(scanner_positions, -40, 40)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, 40, 40)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_xDist(scanner_positions, 20, 60)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, -20, 20)

anim = SLAM(room_size_x, room_size_y, obstacles, scanner_positions, False)

In [16]:
#Из за слишком большого количества кадров анимации, вывести видео в colab не получится, только сохранить в файл ((
#anim

In [17]:
writervideo = animation.FFMpegWriter(fps=20)
anim.save('animation_room3(::)_accurate.mp4', writer=writervideo)

In [ ]:
room_size_x = 150
room_size_y = 100
obstacles = []

for i in range(40, 85):
    obstacles.append((i,40))
for i in range(95, 105):
    obstacles.append((i,40))
for i in range(115, 150):
    obstacles.append((i,40))
for i in range(0, 40):
    obstacles.append((i,20))
for i in range(120, 150):
    obstacles.append((i,20))


for i in range(40, 100):
    obstacles.append((40,i))
    obstacles.append((100,i))
for i in range(0, 5):
    obstacles.append((40,i))
    obstacles.append((120,i))
for i in range(15, 25):
    obstacles.append((40,i))
    obstacles.append((120,i))
for i in range(35, 40):
    obstacles.append((40,i))
    obstacles.append((120,i))


for i in range(140, 150):
    for j in range(40, 97):
        obstacles.append((i,j))
for i in range(120, 140):
    for j in range(40, 50):
        obstacles.append((i,j))
for i in range(100, 110):
    for j in range(70, 97):
        obstacles.append((i,j))
for i in range(112, 116):
    for j in range(71, 76):
        obstacles.append((i,j))
        obstacles.append((i,j+10))
        obstacles.append((i,j+20))


for i in range(140, 150):
    for j in range(0, 20):
        obstacles.append((i,j))
for i in range(145, 150):
    for j in range(20, 40):
        obstacles.append((i,j))
for i in range(140, 145):
    for j in range(27, 33):
        obstacles.append((i,j))
for i in range(125, 135):
    for j in range(35, 40):
        obstacles.append((i,j))


for i in range(0, 40):
    for j in range(0, 5):
        obstacles.append((i,j))
        obstacles.append((i,j+15))
for i in range(0, 5):
    for j in range(5, 15):
        obstacles.append((i,j))


for i in range(0, 20):
    for j in range(20, 30):
        obstacles.append((i,j))
for i in range(0, 10):
    for j in range(35, 55):
        obstacles.append((i,j))
for i in range(0, 10):
    for j in range(60, 97):
        obstacles.append((i,j))
for i in range(30, 40):
    for j in range(45, 65):
        obstacles.append((i,j))
        obstacles.append((i,j+30))
for i in range(24, 28):
    for j in range(52, 58):
        obstacles.append((i,j))
        obstacles.append((i,j+30))


for i in range(40, 80):
    for j in range(40, 50):
        obstacles.append((i,j))
for i in range(40, 70):
    for j in range(60, 85):
        obstacles.append((i,j))
for i in range(90, 100):
    for j in range(65, 97):
        obstacles.append((i,j))
for i in range(40, 45):
    for j in range(87, 97):
        obstacles.append((i,j))
for i in range(47, 51):
    for j in range(89, 95):
        obstacles.append((i,j))

scanner_positions = [(60, 1,math.radians(90))]
# Room 0 (60, 10, 0)
scanner_positions = move_on_yDist(scanner_positions, 10, 5)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, -50, 25)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
scanner_positions = move_on_xDist(scanner_positions, 50, 25)
# Room 1 (90, 30, 0)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, 20, 10)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, -30, 15)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, 10, 5)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, -14, 7)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, 56, 28)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
scanner_positions = move_on_yDist(scanner_positions, -56, 28)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, 14, 7)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, -10, 5)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, 60, 30)
# Room 2 (110, 30, 0)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, 24, 12)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, -40, 20)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
scanner_positions = move_on_xDist(scanner_positions, 30, 15)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, 38, 19)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, -24, 12)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
scanner_positions = move_on_xDist(scanner_positions, 24, 12)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, -38, 19)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, 10, 5)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_yDist(scanner_positions, -24, 12)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_xDist(scanner_positions, 20, 10)
# Room 3 (110, 28, 0)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, 30, 15)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_xDist(scanner_positions, 20, 10)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, 36, 18)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
scanner_positions = move_on_yDist(scanner_positions, -36, 18)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-90))
scanner_positions = move_on_xDist(scanner_positions, -20, 10)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, -32, 16)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
# Room 4 (110, 8, 0)
scanner_positions = move_on_xDist(scanner_positions, 20, 10)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
scanner_positions = move_on_xDist(scanner_positions, -20, 10)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, -20, 10)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
# Room 5 (110, 8, 0)
scanner_positions = move_on_xDist(scanner_positions, 20, 10)
scanner_positions = turn_on_angle(scanner_positions, math.radians(-180), 6)
scanner_positions = move_on_xDist(scanner_positions, -70, 35)
scanner_positions = turn_on_angle(scanner_positions, math.radians(90))
scanner_positions = move_on_yDist(scanner_positions, -8, 4)

# True False
anim = SLAM(room_size_x, room_size_y, obstacles, scanner_positions, False)

In [19]:
#Аналогично, слишком много кадров, только файл
#anim

In [20]:
writervideo = animation.FFMpegWriter(fps=60)
anim.save('animation_room4(house).mp4', writer=writervideo)

# **Скан Матчер**

In [ ]:
# DEFINE CONSTANTS
plot_size = (5,5)                           # Размер рисунков при отрисовке
max_depth = 100                             # Максимальная длина луча сканера
angle_step = 5                              # Шаг угла в градусах
ROUND_DECIMALS = 4                          # Для округления чисел с плавающей точкой

## **Тестирование ошибок при небольшом и большом смещении в пространсве и угле**

In [ ]:
#Тестовая комната
room_size_x = 30
room_size_y = 20
obstacles = [(10, 5),(10, 10),(5, 10),(15,10),(10,15)]
RobotPozition = [
    (5,5,0)
    ,(5,5,math.radians(15))
    ,(5,5,0)
    ,(5,5,math.radians(30))
]
RobotPozition1 = [
    (8,8,0)
    ,(8,8,0)
    ,(15,15,0)
    ,(15,15,0)
]

for i in range(len(RobotPozition)):
    print(f"\nТестовая комната: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i], RobotPozition1[i])

Как видно из данного примера, если сканер находится в сложной комнате и перемещается на значительное расстоение и угол, то точность рассчета трансформации резко снижается. Но в реальности данный сценарий мало вероятен посколько сканирование происходит регулярно и робот не успеет проехать такое расстоение как было представлено в данном примере.

## **Тестирование зависимости точности расчета трансформации от количества точек в облаке**

In [ ]:
#Тестовая комната (коллонны + angle_step=10 (36 точек))
angle_step=10
room_size_x = 30
room_size_y = 20
obstacles = [(10, 5),(10, 10),(5, 10),(15,10),(10,15)]
RobotPozition = [
    (5,5,math.radians(0)),
    (5,5,math.radians(90))
]
RobotPozition1 = [
    (5,7,math.radians(0)),
    (5,7,math.radians(90))
]

for i in range(len(RobotPozition)):
    print(f"\nТестовая комната: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i],RobotPozition1[i])

In [ ]:
#Тестовая комната (коллонны + angle_step=5 (72 точек))
angle_step=5
room_size_x = 30
room_size_y = 20
obstacles = [(10, 5),(10, 10),(5, 10),(15,10),(10,15)]
RobotPozition = [
    (5,5,0)
]
RobotPozition1 = [
    (8,8,0)
]

for i in range(len(RobotPozition)):
    print(f"\nТестовая комната: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i],RobotPozition1[i])

In [ ]:
#Тестовая комната (коллонны + angle_step=0.1 (3600 точек))
angle_step=0.1
room_size_x = 30
room_size_y = 20
obstacles = [(10, 5),(10, 10),(5, 10),(15,10),(10,15)]
RobotPozition = [
    (5,5,0)
]
RobotPozition1 = [
    (8,8,0)
]

for i in range(len(RobotPozition)):
    print(f"\nТестовая комната: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i],RobotPozition1[i])

Как видно из данного примера, увеличение количества точек в облаке (уменьшение градуса поворота лазера) до 3600(0.1 градус) не приводит к увеличению точности рассчета трансформации, в то же время слишком малое количество точек (36)(10 градусов) тоже не является достаточным для создания облака точек. Количество точек в 72(5 градусов) было определено как оптимальное в дальнейшие эксперименты будут проводится с данным числом сканирований.

## **Эксперименты**

In [ ]:
# REDEFINE CONSTANTS
plot_size = (5,5)                           # Размер рисунков при отрисовке
max_depth = 100                             # Максимальная длина луча сканера
angle_step = 5                             # Шаг угла в градусах
ROUND_DECIMALS = 4                          # Для округления чисел с плавающей точкой

In [ ]:
#Комната №1 (Имени усталости -_-)
room_size_x = 25
room_size_y = 20
obstacles = [
    (1,15), (2,15), (3,15), (4,15), (5,15), (6,15), (7,15),
    (19,15),(20,15), (21,15), (22,15), (23,15), (24,15), (25,15),
    (8,6),(9,6),(10,6),(11,6),(12,6),(13,6),(14,6),(15,6),(16,6),(17,6),(18,6)
]

RobotPozition = [
    (2,2,0)
    ,(12,10,math.radians(30))
    ,(10,15,0)
    ,(24,18,0)
]
RobotPozition1 = [
    (5,4,0)
    ,(10,11,0)
    ,(8,12,0)
    ,(21,17,0)
]

for i in range(len(RobotPozition)):
    print(f"\nКомната №1: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i],RobotPozition1[i])

In [ ]:
#Комната №2 (Имени грусти ( )
room_size_x = 25
room_size_y = 20
obstacles = [
    (1, 1),(2, 1), (2, 2),(3, 2), (3, 3),(4, 3),(5, 3), (5, 4),(6, 4),(7, 4), (7, 5),(8, 5),(9, 5), (9, 6),(10,6),(11,6), (11,7),(12,7),
    (24,1),(23,1), (23,2),(22,2), (22,3),(21,3),(20,3), (20,4),(19,4),(18,4), (18,5),(17,5),(16,5), (16,6),(15,6),(14,6), (14,7),(13,7),
]

RobotPozition = [
    (8,10,0)
    ,(2,18,math.radians(15))
    ,(10,15,0)
    ,(24,18,0)
]
RobotPozition1 = [
    (10,13,0)
    ,(5,16,0)
    ,(8,12,0)
    ,(21,17,math.radians(15))
]

for i in range(len(RobotPozition)):
    print(f"\nКомната №2: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i],RobotPozition1[i])

In [ ]:
#Комната №3 (Имени непонимания >< )
room_size_x = 30
room_size_y = 30
obstacles = [
    (1, 10),(2, 10),(3, 10), (3, 11),(4, 11),(5, 11), (5, 12),(6, 12),(7, 12), (7, 13),(8, 13),(9, 13), (9, 14),(10,14),(11,14),
    (29,10),(28,10),(27,10), (27,11),(26,11),(25,11), (25,12),(24,12),(23,12), (23,13),(22,13),(21,13), (21,14),(20,14),(19,14),

    (11,15),(12,15),(13,15),

    (1, 20),(2, 20),(3, 20), (3, 19),(4, 19),(5, 19), (5, 18),(6, 18),(7, 18), (7, 17),(8, 17),(9, 17), (9, 16),(10,16),(11,16),
    (29,20),(28,20),(27,20), (27,19),(26,19),(25,19), (25,18),(24,18),(23,18), (23,17),(22,17),(21,17), (21,16),(20,16),(19,16),

    (17,15),(18,15),(19,15)
]

RobotPozition = [
    (8,9,0)
    ,(2,15,math.radians(15))
    ,(15,25,0)
    ,(24,4,0)
    ,(15,10,0)
]
RobotPozition1 = [
    (10,12,0)
    ,(5,16,0)
    ,(15,20,0)
    ,(28,3,math.radians(15))
    ,(15,20,0)
]

for i in range(len(RobotPozition)):
    print(f"\nКомната №3: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i],RobotPozition1[i])

In [ ]:
#Комната №5 (Имени пятерочку хачу 5 )
room_size_x = 40
room_size_y = 20
obstacles = [
    (1, 1),(2, 1),(3, 1),(4, 1),(5, 1),(6, 1),(7, 1),(7, 2),
    (8, 2),(8, 3),(8, 4),(8, 5),(8, 6),(8, 7),(8, 8),(8, 9),(8, 10),
    (7, 10),(7, 11),(6, 11),(5, 11),(4, 11),(3, 11),(2, 11),(1, 11),
    (1, 12),(1, 13),(1, 14),(1, 15),(1, 16),(1, 17),(1, 18),(1, 19),
    (2, 19),(3, 19),(4, 19),(5, 19),(6, 19),(7, 19),(8, 19),
]
for i in range(room_size_y):
    obstacles.append((9,i))

RobotPozition = [
    (2,9,0)
    ,(15,10,math.radians(15))
    ,(35,15,0)
    ,(30,5,0)
]
RobotPozition1 = [
    (5,7,0)
    ,(14,16,0)
    ,(24,14,0)
    ,(34,9,math.radians(80))
]

for i in range(len(RobotPozition)):
    print(f"\nКомната №5: Эксперимент {i+1}\n")
    Scan_experiment(room_size_x, room_size_y, obstacles,RobotPozition[i],RobotPozition1[i])